# Tutorial 04: Finite Elements for the Wave Equation

In this tutorial we solve the wave equation formulated as a first order in time system. This way the example serves as a model for the treatment of systems of partial differential equations in PDELab. This tutorial depends on tutorial 01 and 03.

# PDE Problem

 - [x] subequations Latex - no
 - [x] cover also 1D case
   - [x] maybe vtkwriter needs to be adjusted (vtp instead of vtu) -no changes were necessary
 - [ ] try direct solver
   - [ ] combination of DS + 1D fast enough?
 - [x] directory mit c++ anlegen (statt C) 
   - [x] header in jupyter header includen
 - [x] maybe: replace STRINGIZE makro with std::to_string
 - [ ] make sequenceWriter downloadable as well - compressable?
 - [ ] nicer citations? extension `sphinxcontrib.bibtex`

 Discuss:
 - [ ] enable  ENABLE_SUITESPARSE in CMake
 - [ ] no if/else in create directory, because function will give error on failure
 - [ ] Append Atributes Filter

$$\require{amsmath}$$

As an example for a system we consider the wave equation with reflective boundary conditions:
}
\begin{align} \label{eq:WaveEquation}
\partial_{tt} u-c^2\Delta u  &= 0 &&\text{in $\Omega\times\Sigma$},\\
u &= 0 &&\text{on $\partial\Omega$},\\
u &= q &&\text{at $t=0$},\\
\partial_t u &= w &&\text{at $t=0$},
\end{align}


where $c$ is the speed of sound.
Renaming $u_0=u$ and introducing $u_1=\partial_t u_0 =\partial_t u$ we can write the wave equation as a system of two equations:


\begin{align} 
\partial_t u_1 - c^2\Delta u_0 &=0 &&\text{in $\Omega\times\Sigma$}, \label{eq:2a}\\
\partial_t u_0 - u_1 &= 0 &&\text{in $\Omega\times\Sigma$},  \label{eq:2b}\\
u_0 &= 0 &&\text{on $\partial\Omega$},\\
u_1 &= 0 &&\text{on $\partial\Omega$},\\
u_0 &= q &&\text{at $t=0$},\\
u_1 &= w &&\text{at $t=0$}.\label{eq:2f}
\end{align}

Since $u_0=u=0$ on the boundary we also have $\partial_t u = u_1 = 0$ on the boundary.
But one may also omit the boundary condition on $u_1$.

Note that there are several alternative ways how to write the scalar equation
\eqref{eq:WaveEquation} as a system of PDEs:

1. [Eriksson et al.](#cit1) apply the Laplacian to equation \eqref{eq:2b}
\begin{equation}  \label{eq:Eriksson}
\Delta \partial_t u_0 - \Delta u_1 = 0
\end{equation} 
which has advantages for energy conservation but requires additional smoothness properties.
2. Alternatively, we may introduce the abbreviations $q=\partial_t u$ and $w=-\nabla u$, so $\partial_{tt} u - c^2 \Delta u =\partial_{tt} u - c^2 \nabla\cdot\nabla u = \partial_{t} q + c^2 \nabla\cdot w = 0$. Taking partial derivatives of the introduced variables we obtain $\partial_{x_i} q=\partial_{x_i} \partial_t u = \partial_t \partial_{x_i}  u = - \partial_t w_i$. This results in a first-order hyperbolic system of PDEs for $q$ and $w$
\begin{align*}
\partial_t q + c^2 \nabla\cdot w &= 0\\
\partial_t w + \nabla q &= 0
\end{align*}
which are called equations of linear acoustics. This formulation is physically more relevant. It can be modified to handle discontinuous material properties and upwind finite volume methods can be used for numerical treatment.

Here we will stay, however, with the simplest formulation \eqref{eq:2a} - \eqref{eq:2f} for simplicity.

## Weak Formulation

Multiplying \eqref{eq:2a}
with the test function $v_0$ and \eqref{eq:2b} with the test function $v_1$
and using integration by parts we arrive at the weak formulation: Find $(u_0(t),u_1(t))\in
U_0\times U_1$ s.t.
\begin{align}
  d_t (u_1,v_0)_{0,\Omega} + c^2 (\nabla u_0, \nabla v_0)_{0,\Omega} &=
  0 \quad \forall v_0 \in U_0 \notag \\
  d_t (u_0,v_1)_{0,\Omega} - (u_1,v_1)_{0,\Omega} &= 0 \quad \forall
  v_1 \in U_1 \label{eq:WeakFormSystem}
\end{align}
where we used the notation of the $L^2$ inner product $(u,v)_{0,\Omega} = \int_\Omega
u v \, dx$. An equivalent formulation to (\ref{eq:WeakFormSystem}) that hides the system structure reads as follows:
\begin{equation}
\label{eq:WeakForm}
\begin{split}
d_t &\left[ (u_0,v_1)_{0,\Omega} + (u_1,v_0)_{0,\Omega}\right] \\
&\hspace{20mm}+ \left[ c^2 (\nabla u_0,\nabla v_0)_{0,\Omega} -(u_1,v_1)_{0,\Omega} \right] = 0
\quad \forall (v_0,v_1)\in U_0\times U_1
\end{split}
\end{equation}
With the latter we readily identify the temporal and spatial residual forms:
\begin{align}
m^{\text{WAVE}}((u_0,u_1),(v_0,v_1)) &= (u_0,v_1)_{0,\Omega} + (u_1,v_0)_{0,\Omega},
\label{eq:TemporalResForm}\\
r^{\text{WAVE}}((u_0,u_1),(v_0,v_1)) &= c^2 (\nabla u_0,\nabla
v_0)_{0,\Omega} - (u_1,v_1)_{0,\Omega} \; , \label{eq:SpatialResForm}
\end{align}
while with the former the system structure is more visible which might help to understand the implementation presented in section
[Realization in PDELab](#implementation).
The spaces $U_0$ and $U_1$ can differ as different types of boundary conditions can be incorporated into the ansatz spaces. But here both spaces are constrained by homogeneous Dirichlet boundary conditions.

## Generalization

The abstract setting of PDELab with its weighted residual formulation carries over to the case of systems of
partial differential equations when cartesian products of functions spaces are introduced, i.e. the abstract *stationary* problem then reads
\begin{equation}
\text{Find $u_h\in U_h=U_h^1\times \ldots \times U_h^s$ s.t.:} \quad r_h(u_h,v)=0
\quad \forall v\in V_h=V_h^1\times\ldots\times V_h^s
\label{Eq:BasicSystemBuildingBlock}
\end{equation}
with $s$ the number of components in the system. Again the concepts are completely orthogonal meaning that $r_h$ might be affine linear or nonlinear in its first argument and the instationary case works as well.

From an organizational point of view it makes sense to allow that a component space $U_h^i$ in the cartesian product is itself a product space. This naturally leads to a *tree structure* in the
function spaces.

Consider as an example the Stokes equation in $d$ space dimensions.There one has pressure $p$ and velocity $v$ with components $v_1,\ldots,v_d$ as unknowns. An appropriate function space then would be $$ U = (P,(V^1,\ldots,V^d)).$$

# Finite Element Method

The finite element method applied to \eqref{eq:WeakForm} is straightforward. We may use the conforming space $V_h^{k,d}(\mathcal{T}_h)$ of degree $k$ in dimension $d$ for each of the components. Typically one would choose
the same polynomial degree for both components.

# Realization in PDELab
<a id= "implementation"> </a>

In [ ]:
//#define ENABLE_SUITESPARSE true

In [ ]:
#include<dune/jupyter.hh>
#include "wavefem.hh"

In [ ]:
// open ini file
Dune::ParameterTree ptree;
Dune::ParameterTreeParser ptreeparser;
ptreeparser.readINITree("exercise04.ini",ptree);

// read ini file
const int refinement = ptree.get<int>("grid.refinement");

Choose polynomial degree:

In [ ]:
const int degree = 2;

Instantiation of a 1D grid, which allows for a faster execution time compared to the 2D grid.

In [ ]:

static const int dim = 1;
// read grid parameters from input file
using DF = Dune::OneDGrid::ctype;
auto a = ptree.get<DF>("grid.oned.a");
auto b = ptree.get<DF>("grid.oned.b");
auto N = ptree.get<unsigned int>("grid.oned.elements");

// create equidistant intervals
using Intervals = std::vector<DF>;
Intervals intervals(N+1);
for(unsigned int i=0; i<N+1; ++i)
intervals[i] = a + DF(i)*(b-a)/DF(N);

// Construct grid
using Grid = Dune::OneDGrid;
Grid grid(intervals);
grid.globalRefine(refinement);

// call generic function
using GV = Dune::OneDGrid::LeafGridView;
GV gv = grid.leafGridView();


Instantiate a 2D UGGrid, as used in previous tutorials.

In [ ]:
/*
static const int dim = 2;
using Grid = Dune::UGGrid<dim>;
using DF = Grid::ctype;
Dune::FieldVector<DF,dim> L;
//upper right
L[0] = ptree.get("grid.structured.LX",(double)1.0);
L[1] = ptree.get("grid.structured.LY",(double)1.0);
std::array<unsigned int,dim> N;
N[0] = ptree.get("grid.structured.NX",(unsigned int)10);
N[1] = ptree.get("grid.structured.NY",(unsigned int)10);
//lower left
Dune::FieldVector<double,dim> lowerleft(0.0);

// build a structured simplex grid
auto gridp = Dune::StructuredGridFactory<Grid>::createSimplexGrid(lowerleft, L, N);

gridp->globalRefine(refinement);
using GV = Grid::LeafGridView;
GV gv=gridp->leafGridView();
*/

In [ ]:
using FEM = Dune::PDELab::PkLocalFiniteElementMap<GV,DF,double,degree>;
FEM fem(gv);

There are several changes now in the driver due to the system of PDEs.The first step is to set up the grid function space using the given finite element map:

In [ ]:
using RF = double;

// Make grid function space used per component
using CON = Dune::PDELab::ConformingDirichletConstraints;
using VBE0 = Dune::PDELab::ISTL::VectorBackend<>;
using GFS0 = Dune::PDELab::GridFunctionSpace<GV,FEM,CON,VBE0>;
GFS0 gfs0(gv,fem);

In [ ]:
// Make grid function space for the system
using VBE =
  Dune::PDELab::ISTL::VectorBackend<
    Dune::PDELab::ISTL::Blocking::fixed
  >;
using OrderingTag = Dune::PDELab::EntityBlockedOrderingTag;
using GFS =
  Dune::PDELab::PowerGridFunctionSpace<GFS0,2,VBE,OrderingTag>;
GFS gfs(gfs0);

- *exercise 3:*

In [ ]:
/*
using VBE = Dune::PDELab::ISTL::VectorBackend <
              Dune::PDELab::ISTL::Blocking::none
            >;
*/

The next step is to set up the product space containing two components. This is done by the following code section:

PDELab offers two different class templates to build product spaces. The one used here is `PowerGridFunctionSpace` which creates a product of a compile-time given number (2 here)
of *identical* function spaces (`GFS0` here) which may only differ in the constraints. With the class template`CompositeGridFunctionSpace` you can create a product space where all components might be different spaces.
We also have to set up names for the child spaces to facilitate VTK output later on:

In [ ]:
using namespace Dune::TypeTree::Indices;
gfs.child(_0).name("u0");
gfs.child(_1).name("u1");

<a id="ordering"> </a>

An important aspect of product spaces is the ordering of the corresponding degrees of freedom. Often the solvers need to exploit an underlying block structure of the matrices.

This works in two stages: An ordering has first to be specified when creating product spaces
which is then subsequently exploited in the backend. Here we use the `EntityBlockedOrderingTag` to specify that all degrees of freedom related to a geometric entity should be numbered consecutively in the coefficient vector. Other options are the `LexicographicOrderingTag` ordering first all degrees of freedom of the first component space, then all of the second component space and so on. With the Iterative Solver Template Library ISTL it is now possible to exploit the block structure at compile-time.
Here we use the tag `fixed` in the ISTL vector backend to indicate that at this level we want to create blocks of fixed size (in this case the block size will be two --
corresponding to the degrees of freedom per entity). Another option would be the tag `none` which is the default. Then the degrees of freedom are still ordered in the specified way but no block structure is introduced on the ISTL level. *Important notice:* Using fixed block structure in ISTL requires that there is the same number of degrees of freedom per entity. This is true for polynomial degrees one and two but *not* for higher polynomial degree!

In order to define a function that specifies the initial value we can use the same techniques as in the scalar case. A PDELab grid function
is be constructed from the lambda closure


In [ ]:
auto u = Dune::PDELab::makeGridFunctionFromCallable(
    gv,
    [](const auto& x){
      Dune::FieldVector<RF,dim> rv(0.0);
      for (int i=0; i<dim; i++) rv[0] += (x[i]-0.375)*(x[i]-0.375);
      rv[0] = std::max(0.0,1.0-8.0*sqrt(rv[0]));
      return rv;
   }
);;

The lambda closure is now returning two components in a `FieldVector`. The first component is the initial value for $u$ and the second component
is the initial value for $\partial_t u$. 

Using the grid function a coefficient vector can now be initialized:

In [ ]:
using Z = Dune::PDELab::Backend::Vector<GFS,RF>;
Z z(gfs); // initial value
Dune::PDELab::interpolate(u,gfs,z);

The next step is to assemble the constraints container for the composite function space. Unfortunately there is currently no way to define the constraints for both components in one go. We need to set up a separate lambda closure for each component:

In [ ]:
auto b0 = Dune::PDELab::
  makeBoundaryConditionFromCallable(
    gv,
    [](const auto& x){return true;} 
  );;

In [ ]:
auto b1 = Dune::PDELab::
  makeBoundaryConditionFromCallable(
    gv,
    [](const auto& x){return true;}
  );;

and then combine it using:

In [ ]:
using B = Dune::PDELab::CompositeConstraintsParameters<
  decltype(b0),decltype(b1)
  >;
B b(b0,b1);

Note that you could define different constraints for each component space although it is the same underlying function space. 
Now the constraints container can be assembled as before:

In [ ]:
using CC = typename GFS::template ConstraintsContainer<RF>::Type;
CC cc;
Dune::PDELab::constraints(b,gfs,cc);

In [ ]:
std::cout << "constrained dofs=" << cc.size() << " of "
          << gfs.globalSize() << std::endl;
set_constrained_dofs(cc,0.0,z); // set zero Dirichlet boundary conditions

Then the VTK writer is prepared and the first file is written.

In [ ]:
int subsampling=ptree.get("output.subsampling",(int)1);
using VTKWRITER = Dune::SubsamplingVTKWriter<GV>;
VTKWRITER vtkwriter(gv,Dune::refinementIntervals(subsampling));
std::string filename=ptree.get("output.filename","output") + std::to_string(dim) + "d";

std::filesystem::create_directory(filename);

using VTKSEQUENCEWRITER = Dune::VTKSequenceWriter<GV>;
VTKSEQUENCEWRITER vtkSequenceWriter(
  std::make_shared<VTKWRITER>(vtkwriter),filename,filename,"");

As we do not want to manually extract the subspaces for $u_0$ and $u_1$ from the overall space to add to them to the VTK writer, we call a PDELab helper function that handles this automatically:

In [ ]:
// add data field for all components of the space to the VTK writer
Dune::PDELab::addSolutionToVTKWriter(vtkSequenceWriter,gfs,z);

The rest of the is the same as for tutorial 03 except that a linear solver is used instead of Newton's method.

In [ ]:
vtkSequenceWriter.write(0.0,Dune::VTK::appendedraw);

// Make instationary grid operator
double speedofsound=ptree.get("problem.speedofsound",(double)1.0);
using LOP = WaveFEM<FEM>;
// using LOP = WaveFEMElip<FEM>;
LOP lop(speedofsound);
using TLOP = WaveL2<FEM>;
// using TLOP = WaveElip<FEM>;
TLOP tlop;
using MBE = Dune::PDELab::ISTL::BCRSMatrixBackend<>;
int degree = ptree.get("fem.degree",(int)1);
MBE mbe((int)pow(1+2*degree,dim));
using GO0 = Dune::PDELab::GridOperator<GFS,GFS,LOP,MBE,RF,RF,RF,CC,CC>;
GO0 go0(gfs,cc,gfs,cc,lop,mbe);
using GO1 = Dune::PDELab::GridOperator<GFS,GFS,TLOP,MBE,RF,RF,RF,CC,CC>;
GO1 go1(gfs,cc,gfs,cc,tlop,mbe);
using IGO = Dune::PDELab::OneStepGridOperator<GO0,GO1>;
IGO igo(go0,go1);

// Linear problem solver
//--iterative solver
using LS = Dune::PDELab::ISTLBackend_SEQ_BCGS_SSOR;
LS ls(5000,false);
using SLP = Dune::PDELab::StationaryLinearProblemSolver<IGO,LS,Z>;
SLP slp(igo,ls,1e-8);
//--direct solver
//using LDS = Dune::PDELab::ISTLBackend_SEQ_UMFPack;
//LDS lds;
//using SLP = Dune::PDELab::StationaryLinearProblemSolver<IGO,LDS,Z>;
//SLP slp(igo,lds,1e-8);
//---

// select and prepare time-stepping scheme
int torder = ptree.get("fem.torder",(int)1);
Dune::PDELab::OneStepThetaParameter<RF> method1(1.0);
Dune::PDELab::Alexander2Parameter<RF> method2;
Dune::PDELab::Alexander3Parameter<RF> method3;
Dune::PDELab::TimeSteppingParameterInterface<RF>* pmethod=&method1;
if (torder==1) pmethod = &method1;
if (torder==2) pmethod = &method2;
if (torder==3) pmethod = &method3;
if (torder<1||torder>3) std::cout<<"torder should be in [1,3]"<<std::endl;
Dune::PDELab::OneStepMethod<RF,IGO,SLP,Z,Z> osm(*pmethod,igo,slp);
osm.setVerbosityLevel(2);

// subspaces
using U0SUB = Dune::PDELab::GridFunctionSubSpace<GFS,Dune::TypeTree::TreePath<0> >;
U0SUB u0sub(gfs);
using U1SUB = Dune::PDELab::GridFunctionSubSpace<GFS,Dune::TypeTree::TreePath<1> >;
U1SUB u1sub(gfs);

// Make discrete grid functions for components
using U0DGF = Dune::PDELab::DiscreteGridFunction<U0SUB,Z>;
U0DGF u0dgf(u0sub,z);
using U1DGF = Dune::PDELab::DiscreteGridFunction<U1SUB,Z>;
U1DGF u1dgf(u1sub,z);

// initialize simulation time
RF time = 0.0;

In [ ]:
// time loop
RF T = ptree.get("problem.T",(RF)1.0);
RF dt = ptree.get("fem.dt",(RF)0.1);
while (time<T-1e-8)
{
  // do time step
  Z znew(z);
  osm.apply(time,dt,z,znew);

  // accept time step
  z = znew;
  time+=dt;

  //exercise 5: put your code here

  // output to VTK file
  vtkSequenceWriter.write(time,Dune::VTK::appendedraw);
}

In [ ]:
Dune::TestStruct<GV> test1 = {vtkSequenceWriter,filename};

In [ ]:
test1

# Execution Times

dim | solver  | dt | degree | torder |T_tot   | *duration*|
----|---------|----|--------|--------|------------|-----------|
1   |iterative|0.01|  2.0   |   2    |   3        |**1min40s**|
2   |    -"-  | -"-|   -"-  |   -"-  |     -"-     |**>30min**|

# Local Operator

## Spatial Local Operator

The spatial residual form \eqref{eq:SpatialResForm} is implemented by the local operator `WaveFEM` in file `wavefem.hh`. Cache construction and flags settings are the same as in tutorial 01 and 03. Only volume terms are used here. Note also that no parameter object is necessary as the only parameter is the speed of sound $c$.

### `alpha_volume` Method

The method `alpha_volume` has the *same* interface as in the scalar case:

```c++
template<typename EG, typename LFSU, typename X, typename LFSV, typename R>
  void alpha_volume (const EG& eg, const LFSU& lfsu, const X& x, const LFSV& lfsv, R& r) const
```
However the trial and test function spaces `LFSU` and `LFSV` now reflect the component structure of the global function space, i.e. they consist of two components. *Important notice: Here we assume that trial and test space are identical (up to constraints) and that also both components are identical!*
The two components can be extracted with the following code
```c++
// select the two components (but assume Galerkin scheme U=V)
using namespace Dune::TypeTree::Indices;
auto lfsu0 = lfsu.child(_0);
auto lfsu1 = lfsu.child(_1);
```
The function spaces `lfsu0` and `lfsu1` are now scalar spaces (which we assume to be identical).

After extracting the dimension
```c++
const int dim = EG::Entity::dimension;
```
we select a quadrature rule
```c++
auto geo = eg.geometry();
const int order = 2*lfsu0.finiteElement().localBasis().order();
auto rule = Dune::PDELab::quadratureRule(geo,order);
```
and may now loop over the quadrature points.
For each quadrature point, evaluate the basis function of the first component:    
```c++   
for (const auto& ip : rule)
  {
    auto& phihat = 
    cache.evaluateFunction(ip.position(),lfsu0.finiteElement().localBasis());
```
As the components are identical we need only evaluate the basis once and can compute the value of $u_1$ at the quadrature point
```c++
    // evaluate u1
    RF u1=0.0;
    for (std::size_t i=0; i<lfsu0.size(); i++) u1 += x(lfsu1,i)*phihat[i];
```
Then we evaluate the gradients of the basis functions
```c++
    auto& gradphihat = 
    cache.evaluateJacobian(ip.position(),lfsu0.finiteElement().localBasis());
```        
transform them from the reference element to the real element
```c++
    const auto S = geo.jacobianInverseTransposed(ip.position());
    auto gradphi = makeJacobianContainer(lfsu0);
    for (std::size_t i=0; i<lfsu0.size(); i++)
       S.mv(gradphihat[i][0],gradphi[i][0]);
```         
and compute the gradient of $u_0$:
```c++
    Dune::FieldVector<RF,dim> gradu0(0.0);
    for (std::size_t i=0; i<lfsu0.size(); i++)
      gradu0.axpy(x(lfsu0,i),gradphi[i][0]);
```          
With the integration factor
```c++
    RF factor = ip.weight() * geo.integrationElement(ip.position());
 ```   
the residuals can now be accumulated:    
```c++
   for (std::size_t i=0; i<lfsu0.size(); i++) {
     r.accumulate(lfsu0,i,c*c*(gradu0*gradphi[i][0])*factor);
     r.accumulate(lfsu1,i,-u1*phihat[i]*factor);
```

### `jacobian_volume` Method

As the problem is linear it is advisable to also implement the `jacobian_volume` method for efficiency and accuracy.
The interface is the same as in the scalar case:
```c++
template<typename EG, typename LFSU, typename X, typename LFSV, typename M>
void jacobian_volume (const EG& eg, const LFSU& lfsu, const X& x, const LFSV& lfsv,
                      M& mat) const
```                      
Component selection, quadrature rule selection and basis evaluation are the same as in `alpha_volume`.
We only consider the accumulation of the Jacobian entries here:
```c++
// integrate both equations
RF factor = ip.weight() * geo.integrationElement(ip.position());
for (std::size_t j=0; j<lfsu0.size(); j++)
  for (std::size_t i=0; i<lfsu0.size(); i++) {
    mat.accumulate(lfsu0,i,lfsu0,j,c*c*(gradphi[j][0]*gradphi[i][0])*factor);
    mat.accumulate(lfsu1,i,lfsu1,j,-phihat[j]*phihat[i]*factor);
  }
```  
Note how the diagonal sub-blocks of the Jacobian with respect to the first and second component are accessed. Finally, `WaveFEM` also implements the matrix-free versions
for Jacobian application.  

## Temporal Local Operator

The temporal residual form \eqref{eq:TemporalResForm} is implemented by the local operator `WaveL2` in file `wavefem.hh`. Cache construction and flags settings are the same as in tutorial 01 and 03. Only volume terms are used here.

### `alpha_volume` Method

The `alpha_volume` method is pretty similar
to the one in the spatial operator, except that the value of $u_0$ is needed instead of the gradient. Here we just show the residual accumulation:
```c++
 // integration factor
RF factor = ip.weight() * geo.integrationElement(ip.position());

// integrate u*phi_i
for (std::size_t i=0; i<lfsu0.size(); i++) {
  r.accumulate(lfsu0,i,u1*phihat[i]*factor);
  r.accumulate(lfsu1,i,u0*phihat[i]*factor);
}
```
Note that $u_1$ is integrated with respect to test function $v_0$
and vice versa.

### `jacobian_volume` Method

The corresponding Jacobian entries are accumulated in the `jacobian_volume` method:
```c++
// integration factor
RF factor = ip.weight() * geo.integrationElement(ip.position());

// loop over all components
for (std::size_t j=0; j<lfsu0.size(); j++)
  for (std::size_t i=0; i<lfsu0.size(); i++) {
    mat.accumulate(lfsu0,i,lfsu1,j,phihat[j]*phihat[i]*factor);
    mat.accumulate(lfsu1,i,lfsu0,j,phihat[j]*phihat[i]*factor);
  }
```
That's it! 293 lines of code to implement the finite element method for the wave equation.

# Exercise

## Getting to Know the Code

The code of this exercise solves the wave equation formulated as a first order in time system. As already explained above, we can write the wave equation as a system of two equations by substituting $u_0=u$ and introducing $u_1=\partial_t u_0 =\partial_t u$.
  
As in the previous exercises you can control most of the settings through the ini-file `exercise04.ini`. Get an overview of the configurable settings, compile and run `exercise04`.

The program writes output with the extension `pvd`. This is one of several ways to write VTK output for the instationary case, c.f. the documentation of tutorial03. The `pvd`-file can be visualized by ParaView and consists of a collection of the corresponding `vtu`-files. <font color='red'>In order to visualize a 1D solution, one can apply the "Plot Over Line" filter.</font> Note that our solution is always given by $u_0$.

- *1d solution of wave equation*

<video src="wave1d.ogv" controls width="60%">

- *2d solution of wave equation*

<video src="wave2d.ogv" controls width="100%">

## Try various time integrators, in particular the Crank-Nicolson method

We want to examine the numerical solution under different time discretization schemes, eg. Implicit Euler or Crank-Nicolson. In order to change the time discretization scheme you will have to search for the line
```c++
Dune::PDELab::Alexander2Parameter<RF> pmethod;
```

Recall the previous `exercise03` and change the one step $\theta$ scheme in a way that corresponds to the Crank-Nicolson method.

 **Note that** you can compare the solutions in ParaView. To do that you have to <font color = 'red'>rename the second solution in `exercise04.ini` and use the "Append Attributes" filter </font>
 before "Plot Over Line". Do not forget to change the parameter `torder` in `exercise04.ini` in order to get the correct scheme.

 **Remark** You can decide which dimension to investigate. A 2D simulation will give you nicer pictures at the cost of longer run times.

- *Crank-Nicolson method:*
```c++
 Dune::PDELab::OneStepThetaParameter<RF> method1(0.5);
```

<video src="wave1dCNIE.ogv" controls width="60%">

## Explore polynomial degrees greater than $2$ by changing the blocking to`none`.

**Step 1:** Find the place where your Local Finite Element Maps are created and make it possible to use polynomial degree 3

```c++
Dune::PDELab::PkLocalFiniteElementMap<GV,DF,double,deg> FEM;
```

Compile your program and check the results.

**Note that** `deg` is a static template parameter.

 - *Program should compile and give a segmentation error when started.*
 - *For the notebook: The initialization of the coefficient vector fails.*


**Step 2:**

Before we start read the part that describes the specification of an ordering when creating product spaces ([here](#ordering)). Using fixed block structure in ISTL requires that the number of degrees of freedom per entity is constant for each geometry type. This is true for polynomial degrees one and two but not for higher polynomial degree!

To avoid segfaults you need to change
```c++
Dune::PDELab::istl::VectorBackend<Dune::PDELab::istl::Blocking::fixed> VBE;
```
  to
```c++
Dune::PDELab::istl::VectorBackend<Dune::PDELab::istl::Blocking::none> VBE;
```
within the notebook.

## Changing the Local Operator

 Now consider the elliptic projection as in Eriksson et al. \cite{Eriksson}, namely applying the Laplacian to equation \eqref{eq:2b}
  \begin{equation}\label{eq:elip}
    -\Delta \partial_t u_0 + \Delta u_1 = 0,
  \end{equation}
which has the advantage of energy conservation but requires additional smoothness properties.

The main work is now to change the local operators given in the file `wavefem.hh`. This is done in several steps:

1. Copy the file `wavefem.hh` to a new file and rename it.
2. Rename the local operators in the new file, e.g. change`WaveL2` to `WaveElip`.
3. Include your new file in `exercise04.cc` and change the types of `LOP` and `TLOP` in
   the notebook.
```c++
// Make instationary grid operator
double speedofsound = ptree.get("problem.speedofsound",1.0);
typedef WaveFEM<FEM> LOP;
LOP lop(speedofsound);
typedef WaveL2<FEM> TLOP;
TLOP tlop;
```
After these preparations you need to implement the following change:
$$ \partial_t u_0 - u_1 = 0 \Rightarrow \Delta \partial_t u_0 -
  \Delta u_1 = 0.$$

1. In the spatial local operator, change:
    $$ - u_1 = 0 \Rightarrow -  \Delta u_1 = 0$$

    See how it is done for the $\Delta u_0$, we recall the
    corresponding part of `WaveFEM::alpha_volume()`:
```c++
// integrate both equations
RF factor = ip.weight() * geo.integrationElement(ip.position());
for (size_t i=0; i<lfsu0.size(); i++) {
	r.accumulate(lfsu0,i,c*c*(gradu0*gradphi[i][0])*factor);
	r.accumulate(lfsu1,i,-u1*phihat[i]*factor);
}
```

- *solution for `WaveFEMElip::alpha_volume`*

```c++
...
// compute gradient of u1
Dune::FieldVector<RF,dim> gradu1(0.0);
for (size_t i=0; i<lfsu1.size(); i++)
  gradu1.axpy(x(lfsu1,i),gradphi[i][0]);

// integrate both equations
RF factor = ip.weight() * geo.integrationElement(ip.position());
for (size_t i=0; i<lfsu0.size(); i++) {
  r.accumulate(lfsu0,i,c*c*(gradu0*gradphi[i][0])*factor);
  r.accumulate(lfsu1,i,-(gradu1*gradphi[i][0])*factor);            
```


2. In the temporal local operator, change:
    $$ \partial_t u_0  = 0 \Rightarrow \Delta \partial_t u_0= 0$$

    See how it is done for the $\Delta u_0$, we recall the
    corresponding part of `WaveL2::alpha_volume()`:
```c++
// integrate u*phi_i
for (size_t i=0; i<lfsu0.size(); i++) {
	r.accumulate(lfsu0,i,u1*phihat[i]*factor);
	r.accumulate(lfsu1,i,u0*phihat[i]*factor);
}
```

 - *solution for `WaveElip::alpha_volume`*
 
```c++
// integrate u*phi_i
for (size_t i=0; i<lfsu0.size(); i++) {
  r.accumulate(lfsu0,i,u1*phihat[i]*factor);
  r.accumulate(lfsu1,i,(gradu0*gradphi[i][0])*factor);
```

3. Do not forget to update the `jacobian_volume()` methods accordingly. As the problem is linear, this should not be too difficult.

- *solution for `WaveFEMElip::jacobian_volume`*

```c++
for (size_t i=0; i<lfsu0.size(); i++) {
  mat.accumulate(lfsu0,i,lfsu0,j,c*c*(gradphi[j][0]*gradphi[i]   [0])*factor);
  mat.accumulate(lfsu1,i,lfsu1,j,-(gradphi[j][0]*gradphi[i][0])*factor);
}
 ```

- *solution for `WaveElip::jacobian_volume`*
 
 ```c++
for (size_t i=0; i<lfsu0.size(); i++)
  S.mv(gradphihat[i][0],gradphi[i][0]);
// loop over all components
for (size_t j=0; j<lfsu0.size(); j++)
  for (size_t i=0; i<lfsu0.size(); i++) {
    mat.accumulate(lfsu0,i,lfsu1,j,phihat[j]*phihat[i]*factor);
    mat.accumulate(lfsu1,i,lfsu0,j,(gradphi[j][0]*gradphi[i][0])*factor);
    }
```

## Energy Conservation

 If we multiply \eqref{eq:2a} by $u_1$ and \eqref{eq:elip} by $u_0$ and add, the terms $-(\Delta u_0,u_1 )$ and $(\Delta u_1,u_0 )$ cancel out, leading to the conclusion that the energy $$E(t) = \|u_1\|^2 + \| \nabla u_0\|^2 $$ is constant in time.

Your task is to check the energy conservation for the elliptic projection and Crank-Nicolson in time. You can use the following PDELab utilities:
```c++
Dune::PDELab::SqrGridFunctionAdapter
Dune::PDELab::integrateGridFunction
Dune::PDELab::DiscreteGridFunctionGradient
Dune::PDELab::SqrGridFunctionAdapter
```

If you have problems with this task check the online documentation https://www.dune-project.org/doxygen/pdelab/master/ or the solution.

 - *solution for energy conservation*
 
```c++
// Energy - integrate u square
typedef Dune::PDELab::SqrGridFunctionAdapter<U1DGF> SQRU1DGF;
SQRU1DGF u1sqrdgf(u1dgf);

typename SQRU1DGF::Traits::RangeType sqru1(0.0);
Dune::PDELab::integrateGridFunction(u1sqrdgf, sqru1, 2*degree);

using DGFU0G = Dune::PDELab::DiscreteGridFunctionGradient<U0SUB, Z>;
DGFU0G dgfu0g(u0sub,z);
//gradient square
using SQRDGFU0G = Dune::PDELab::SqrGridFunctionAdapter<DGFU0G>;
SQRDGFU0G sqrdgfu0g(dgfu0g);
//integrate gradient square
typename SQRDGFU0G::Traits::RangeType sqru0g(0.0);
Dune::PDELab::integrateGridFunction(sqrdgfu0g, sqru0g, 2*degree);

std::cout << "::: energy          " << sqru1 + sqru0g << std::endl;
```

## Additional Task

If you are done with these exercises, you can play with the initial conditions. Use the following setting in one space dimension:

 - Change the initial conditions to $\sin(2x)$ (implement it as a lambda function)
 - Change the domain size to $[0,\pi]$

# Bibliographie

 [1] K. Eriksson and D. Estep and P. Hansbo and C. Johnson. *Computational Differential Equations*. Cambridge University Press, 1996. <a id ="cit1"> </a>

[2] P. G. Ciarlet. *The finite element method for elliptic problems*. SIAM, Classics in Applied Mathematics, 2002.

[3] D. Braess. *Finite Elemente*. Springer, 3. edition, 2003.

[4] S. C. Brenner and L. R. Scott. *The mathematical theory of finite element methods*, Springer, 1994.

[5] H. Elman, D. Silvester and A. Wathen. *Finite Elements and Fast Iterative Solvers*. Oxford University Press,2005.

[6] C. Großmann and H.-G. Roos.*Numerische Behandlung partieller Differentialgleichungen*. Teubner, 2006.

[7] W. Hackbusch. *Theorie und Numerik elliptischer Differentialgleichungen*. Teubner, 1986. http://www.mis.mpg.de/preprints/ln/lecturenote-2805.pdf.

[8] R. Rannacher. *Einführung in die Numerische Mathematik II (Numerik partieller Differentialgleichungen)*. Heidelberg University, 2006. http://numerik.iwr.uni-heidelberg.de/~lehre/notes.

[9] P. Bastian. *Lecture Notes on Scientific Computing with Partial Differential Equations*. Heidelberg University, 2014. http://conan.iwr.uni-heidelberg.de/teaching/numerik2_ss2014/num2.pdf.